<a href="https://colab.research.google.com/github/Paolo20224/Prueba/blob/main/M%C3%A9tricas_de_regresi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Como parte inicial, descargamos la informacion y la cargamos en nuestro cuader en Colab
import pandas as pd
filename = "/content/pokemon.csv"
df = pd.read_csv(filename)